# Moderador de contenidos de video ⏯️ 🔫.

En este notebook aprenderas a crear un moderador de contenido de video


## Este ejemplo vas a usar 2servicios de AI/ML que puedes emplear como llamadas de API: 

- [Amazon Rekognition](https://aws.amazon.com/es/rekognition/): 
Amazon Rekognition facilita la adición de análisis de imágenes y vídeos a sus aplicaciones. Solo tiene que proporcionar una imagen o un vídeo a la API de Amazon Rekognition y el servicio podrá:

Identifique las etiquetas (objetos, conceptos, personas, escenas y actividades) y el texto

Detectar contenido inapropiado

Proporcione capacidades de análisis facial, comparación de rostros y búsqueda de rostros de alta precisión

[Más información](https://docs.aws.amazon.com/es_es/rekognition/latest/dg/what-is.html)




### Requisitos:
- [Una cuenta de AWS](https://aws.amazon.com/es/free/)
- [Foundational knowledge of Python](https://community.aws/tutorials/build-a-knowledge-base-with-multilingual-q-and-a-gen-ai)

### Configuración previa:
- [Uso de AWS Command Line Interface CLI](https://docs.aws.amazon.com/es_es/cli/latest/userguide/cli-chap-welcome.html)
- [Instalar CLI](https://docs.aws.amazon.com/es_es/cli/latest/userguide/getting-started-install.html)
- [Configurar credenciales de AWS para uso local](https://docs.aws.amazon.com/es_es/cli/latest/userguide/cli-configure-files.html)


## ⏯️ 🍿 Empecemos!! 

### Paso 0: [Instala Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html#install-boto3) [Más información de Boto3](https://aws.amazon.com/es/sdk-for-python/)
Es una librería de python para facilitar la integración con los servicios de AWS (Amazon Web Services)

In [ ]:
### Instala la ultima version de Boto3 via pip: 

!pip install boto3

In [ ]:
# Boto3 session with credentials taken from environtment
import boto3
import os

AWS_ACCESS_KEY_ID = YOU_AWS_ACCESS_KEY_ID
AWS_SECRET_ACCESS_KEY = YOU_AWS_SECRET_ACCESS_KEY
AWS_SESSION_TOKEN= YOU_AWS_SESSION_TOKEN
REGION_NAME = YOU_REGION

my_session = boto3.Session(
    aws_access_key_id     = AWS_ACCESS_KEY_ID,
    aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
    aws_session_token     = AWS_SESSION_TOKEN,
    region_name=REGION_NAME
)


### Paso 1: API para moderación de Video. 

> 🚨Nota: Cada servicio tiene un cliente que debes inicializar antes de hacer cada llamada de API. 

Este paso lo realizamos empleando la llamda de API [start_content_moderation](https://boto3.amazonaws.com/v1/documentation/api/1.9.42/reference/services/rekognition.html#Rekognition.Client.start_content_moderation) de [Boto3 Rekognition client](https://boto3.amazonaws.com/v1/documentation/api/1.9.42/reference/services/rekognition.html#client).

### Amazon transcribe toma el video desde un bucket de [Amazon S3](https://docs.aws.amazon.com/s3/index.html), por lo que debes crearlo primero y luego subir el video. [Sigue los pasos decomo crear un Bucket de S3 aca](https://docs.aws.amazon.com/es_es/AmazonS3/latest/userguide/creating-bucket.html)

In [ ]:
rekognition_client = my_session.client('rekognition')

def crea_subitulos(s3bucket,filename):

    startModerationLabelDetection = rekognition_client.start_content_moderation(
                Video={'S3Object': {
                        'Bucket': s3bucket, 
                        'Name': filename}
                    })

    moderationJobId = startModerationLabelDetection['JobId']
    print("Job Id: {0}".format(moderationJobId))
    return moderationJobId

### Paso 2: Obtiene la moderación del contenido. 
Antes de iniciar este paso debes esperar que la moderación de contenido este listos, esto dependera de la duración del video. 

Una vez terminada puedes revisar el resultado con la siguiente API [get_content_moderation](https://boto3.amazonaws.com/v1/documentation/api/1.9.42/reference/services/rekognition.html#Rekognition.Client.get_content_moderation). 


In [ ]:
def obtiene_moderacion(moderationJobId):
    getContentModeration = rekognition_client.get_content_moderation(
                JobId=moderationJobId,
                SortBy='TIMESTAMP')
    return getContentModeration

In [ ]:
s3bucket = "you_bucket_name"
filename = "Duro_de_matar_1min.mp4"

In [ ]:
moderationJobId = crea_subitulos(s3bucket,filename)
getContentModeration = obtiene_moderacion(moderationJobId)
getContentModeration

In [ ]:
getContentModeration['ModerationLabels']

In [ ]:
#Guarda la moderación en un json
filename = filename.replace(".mp4", ".json")
with open('data.json', 'w') as file:
    json.dump(getContentModeration["ModerationLabels"], file, indent=4)
    


> Aca puedes aprender más sobre Amazon Rekognition [Code Samples](https://github.com/aws-samples/amazon-rekognition-code-samples)